In [1]:
%pip install groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Instala o pacote "groq" dentro do ambiente executado.

In [ ]:
import os
os.environ['GROQ_API_KEY'] = "Coloque sua chave aqui"

importa a biblioteca padrão e lê a chave que deve ser importada da API adquirida no Groq cloud.

In [3]:
import os

from groq import Groq

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's rapidly evolving natural language processing (NLP) landscape. These models are designed to process and generate human-like text at unprecedented speeds, enabling a wide range of applications and services that rely on text analysis and generation. Here are some reasons highlighting the importance of fast language models:

1. **Real-time Interaction**: Fast language models enable real-time interaction with users, allowing for instantaneous responses to queries, messages, or voice commands. This is particularly essential for applications like chatbots, virtual assistants, and customer service platforms, where timely responses are critical.
2. **Efficient Text Analysis**: Quick language models facilitate the analysis of vast amounts of text data, making it possible to extract insights, identify trends, and detect patterns in real-time. This is beneficial in applications such as sentiment analysis, entity recognition, and topic modeling.
3. **Impr

primeiramente são importados os pacotes necessários, após isso o cliente que faz as requisições para o modelo é criado, usando a conexão com a achave que usamos anteriormente. Depois de toda a etapa de preparação, é solicitado um chat completion, ou seja, pede ao modelo (no caso o llama-3.3-70b-versatile) uma resposta com base em mensagens enviadas, no caso explicar a importância de modelos de linguagem ágeis. E, por fim, imprimir o conteúdo da primeira resposta dada.

In [4]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.messages = []
        if self.system is not None: 
            self.messages.append({"role": "system", "content": self.system})
        
    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result
    
    def execute(self):
        completion = client.chat.completions.create(
            messages=self.messages,
            model="llama-3.3-70b-versatile",
        )
        return completion.choices[0].message.content

Esse trecho de código cria a classe "Agent". O primeiro método é um construtor (__init__) que passa os parâmetros cliente e system para os atributos da classe instanciada, e, se for passado um parâmetro para system, quer dizer que uma informação do papel "system" vai ser passada para o modelo, colocando na lista de messages. O segundo método é um __call__, ou seja, vai ser chamada toda vez que o objeto for chamado também, se tiver uma mensagem, ela vai ser colocada na lista tendo "usuário" como papel, e também usa a função execute, colocando o resultado dessa função no final da lista de mensagens. E na última função, execute, é solicitado um completion ao modelo usado, semelhantemente ao que foi usado anteriomente.

In [5]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

calculate:
e.g. calculate: 4 * 7 / 3
Runs a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary

get_planet_mass:
e.g. get_planet_mass: Earth
returns weight of the planet in kg

Example session:

Question: What is the mass of Earth times 2?
Thought: I need to find the mass of Earth
Action: get_planet_mass: Earth
PAUSE

You will be called again with this:

Observation: 5.972e24

Thought: I need to multiply this by 2
Action: calculate: 5.972e24 * 2
PAUSE

You will be called again with this:

Observation: 1,1944×10e25

If you have the answer, output it as the Answer.

Answer: The mass of Earth times 2 is 1,1944×10e25.

Now it's your turn:
""".strip()

Somente o prompt que irá ser enviado para o modelo para configurar para um loop de ReAct.

In [6]:
#tools
def calculate(operation):
    return eval(operation)
def get_planet_mass(planet) -> float:
    match planet.lower():
        case "earth":
            return 5.972e24
        case "jupiter":
            return 1.898e27
        case "mars":
            return 6.39e23
        case "mercury":
            return 3.285e23
        case "neptune":
            return 1.024e26
        case "saturn":  
            return 5.683e26
        case "uranus":
            return 8.681e25
        case "venus":
            return 4.86e24
        case _:
            raise 0.0

Apenas dá as ferramentas que o prompt anterior disponibilizou, usado um "eval" para executar as expressões algébricas na primeira função, e um switch case para identifcar os planetas e retornar as massas de cada um.

In [7]:
neil_tyson = Agent(client, system_prompt)

cria o agente com o cliente inicializado anteriormente junto com o prompt para ReAct.

In [8]:
result = neil_tyson("What is the mass of the Earth times 5?")
print(result)

Thought: To find the mass of the Earth times 5, I first need to find the mass of the Earth. 

Action: get_planet_mass: Earth
PAUSE


Aqui é possível ver o resultado do que foi construído até então, com toda a conexão com o modelo e toda a lógica de passar a mensagem para esse modelo. De maneira que a resposta do modelo é exatamente da forma que o prompt pediu.

In [9]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

foi usado só para checar os mensagens enviadas.

In [10]:
result = neil_tyson()
print(result)

No vídeo, esse trecho serve para printar o "action" e o "pause", porém, possivelmente por ser um modelo diferente do usado no video (de dois anos atrás). Os outputs da IA foram diferentes, e, nesse código, essa partes foram na primeira mensagem.

In [11]:
observation = get_planet_mass("Earth")
print(observation)

5.972e+24


Só uma demonstração do que seria uma observation, para uso posterior no futuro.

In [13]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: Now that I have the mass of the Earth, I can multiply it by 5 to find the result.

Action: calculate: 5.972e+24 * 5
PAUSE


agora foi mostrada como um output anterior (ne caso a observation) vai ajudar a realizar o próximo loop.

In [14]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

aqui é possível checar novamente quais mensagens foram enviadas para o modelo.

In [15]:
result = neil_tyson()
print(result)

Novamente no video os passos de action e pause era para ter sido feitos nesse trecho porém já foram feitos antes.

In [16]:
observation = calculate("3.285e23 * 5")
print(observation)

1.6425e+24


In [18]:
next_prompt = f"Observation: {observation}"
result = neil_tyson(next_prompt)
print(result)

Thought: The calculation has been performed, and I now have the result of the Earth's mass times 5.

However, the result seems to be in a different format than I expected, it appears there might have been an error in the exponent. Let me try to re-calculate.

Action: calculate: 5.972e24 * 5
PAUSE


Aqui, a observação anterior foi usada para alimentar esse passo, e com isso foi possível chegar em uma resposta. Além disso, como previsto um dos maiores benefícios em usar sistemas baseados em agentes é a capacidade de revisão de uma resposta, como é possível ver agora.

In [19]:
neil_tyson.messages

[{'role': 'system',
  'content': "You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\ncalculate:\ne.g. calculate: 4 * 7 / 3\nRuns a calculation and returns the number - uses Python so be sure to use floating point syntax if necessary\n\nget_planet_mass:\ne.g. get_planet_mass: Earth\nreturns weight of the planet in kg\n\nExample session:\n\nQuestion: What is the mass of Earth times 2?\nThought: I need to find the mass of Earth\nAction: get_planet_mass: Earth\nPAUSE\n\nYou will be called again with this:\n\nObservation: 5.972e24\n\nThought: I need to multiply this by 2\nAction: calculate: 5.972e24 * 2\nPAUSE\n\nYou will be called again with this:\n\nObservation: 1,1944×10e25\n\nIf you have the a

O resumo de todas as entradas e saídas feitas manualmente até chegar em uma resposta. E agora para evitar todo esse trabalho manual, o a função fundamental para a eficiência da ferramenta, o loop, vai ser implementada.

In [25]:
import re

def agent_loop(max_iterations, system, query):
    agent = Agent(client, system)
    tools = ['calculate', 'get_planet_mass']
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "PAUSE" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            arg = action[0][1]

            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            
            else:
                next_prompt = "Observation: tool not found"

            print(next_prompt)
            continue

        if "Answer" in result:
            break
                





Esse trecho de código mais complexo é a aplicação do loop do ReAct, de maneira que o agente é instanciado já com o prompt que descreve o ReAct e com o máximo de iterações, para evitar um loop infinito e também são determinadas as ferramentas disponíveis (funções) para evitar error. Então é feito um laço while que repete até o número máximo de iterações ou um output do modelo conter "Answer" (na segunda condição). A primeira condição é importante para manter o a persistência do passo anterior do ReAct para o passo atual, de forma que quando for identificado o "pause" e "action", a ferramenta e o o argumento são selecionados, colocados em uma "observation" que, por sua vez, é aplicado ao próximo prompt.

In [26]:
agent_loop(max_iterations=10, system=system_prompt, query="What is the mass of the Earth plus the mass of Mercury, and all of it times 5?")

Thought: To solve this problem, I need to find the masses of Earth and Mercury, add them together, and then multiply the result by 5. First, I'll find the mass of Earth.

Action: get_planet_mass: Earth
PAUSE
Observation: 5.972e+24
Thought: Now that I have the mass of Earth, I need to find the mass of Mercury.

Action: get_planet_mass: Mercury
PAUSE
Observation: 3.285e+23
Thought: I now have the masses of Earth and Mercury. I'll add them together to find the total mass.

Action: calculate: 5.972e+24 + 3.285e+23
PAUSE
Observation: 6.300500000000001e+24
Thought: Now that I have the total mass of Earth and Mercury, I need to multiply it by 5 to get the final result.

Action: calculate: 6.300500000000001e+24 * 5
PAUSE
Observation: 3.1502500000000004e+25
Thought: I have now calculated the total mass of Earth and Mercury, and then multiplied it by 5, so I have the final answer.

Answer: The mass of the Earth plus the mass of Mercury, and all of it times 5 is 3.1502500000000004e+25.


Por fim, a execução do loop, onde vemos cada passo presente na tarefa atribuída, possibilitando ao modelo pensar em um plano e solucionar a tarefa separadamente, aumentando a qualidade da resposta.